# 1. 문제정의

질문1 : 어떤 요소가 앱의 점수에 큰 영향을 주는가?

질문2 : 점수가 높은 앱의 특징은 무엇이고 점수가 낮은 앱의 특징은 무엇인가?

# 2. 방법론

## 2.1. 분석과정

이 질문에 대한 해답을 얻기 위해 아래와 '해석 가능한 머신러닝(xAI)'를 활용한다.

### 2.1.1. 프로세스

1단계 : 데이터 전처리와 EDA를 실시한다.<br>
2단계 : 머신러닝 모델을 만든다.<br>
3단계 : shap value를 통해 어떤 변수가 점수에 가장 큰 영향을 주는지 알아본다.<br>
4단계 : 점수가 높은 앱의 특징과 낮은 앱의 특징을 살펴본다.<br>
5단계 : 최종적으로 앱의 점수를 높이기 위한 방안들을 제시한다.

## 2.1. 데이터 불러오기

In [106]:
import pandas as pd
import numpy as np
import os

In [107]:
os.chdir(r'C:\Users\hjb38\Documents\데이터 분석 과정\data\googleplaystore_data')

In [108]:
data = pd.read_csv('googleplaystore.csv')

In [109]:
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


## 2.2. 데이터 전처리

### 2.2.1. 결측치 제거

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [111]:
data = data.dropna(axis=0, how='any')

In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9360 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             9360 non-null   object 
 1   Category        9360 non-null   object 
 2   Rating          9360 non-null   float64
 3   Reviews         9360 non-null   object 
 4   Size            9360 non-null   object 
 5   Installs        9360 non-null   object 
 6   Type            9360 non-null   object 
 7   Price           9360 non-null   object 
 8   Content Rating  9360 non-null   object 
 9   Genres          9360 non-null   object 
 10  Last Updated    9360 non-null   object 
 11  Current Ver     9360 non-null   object 
 12  Android Ver     9360 non-null   object 
dtypes: float64(1), object(12)
memory usage: 1023.8+ KB


### 2.2.2. App, Current Ver, Android Ver 제거

In [113]:
# App은 고유한 이름이기에 데이터로 사용하기에 어려움이 잇음으로 제거
data = data.drop(labels = ['App'], axis = 1)

In [114]:
# Current Ver, Android Ver는 앱마다 버전의 기준이 다르며,
# 'Varies with device' 값을 다른 것으로 대체하기 어렵기 때문에
# 해석에 어려움이 있을 것으로 판단 되어 제거.
data = data.drop(labels = ['Current Ver', 'Android Ver'], axis = 1)

### 2.2.3. Reviews를 int형으로 변환

In [115]:
# Reviews 는 1개의 데이터 때문에 object형으로 분류됐지만,
# 결측치로 제거했기에 이제 int형으로 변경할 수 있다.
data['Reviews'] = data['Reviews'].astype(int)

### 2.2.4 Size를 int형으로 변환

In [124]:
data['Size'].head()

0     19000000
1     14000000
2    8.7000000
3     25000000
4    2.8000000
Name: Size, dtype: object

In [125]:
# k를 000으로, M을 000000으로 바꿔준다.
x = '.'
data['Size'] = data['Size'].apply(lambda x : data['Size'].str.replace('M', '00000') 
                                  if x in data['Size']
                                  else replace('M', '000000'))

def :
    if '.' in data['Size'] :
        data['Size'].str.replace('M', '00000') 
    else :
        replace('M', '000000')) 
                                  
                                  
    

NameError: name 'replace' is not defined

In [118]:
data['Size'].head(10)

0     19000000
1     14000000
2    8.7000000
3     25000000
4    2.8000000
5    5.6000000
6     19000000
7     29000000
8     33000000
9    3.1000000
Name: Size, dtype: object

In [234]:
# int 형으로 바꿔준다.
data['Size'] = data['Size'].astype(int)

ValueError: invalid literal for int() with base 10: '8.7000000'

In [33]:
data['Size'].value_counts()

Varies with device    1637
14M                    165
12M                    161
15M                    159
11M                    159
                      ... 
459k                     1
186k                     1
143k                     1
350k                     1
454k                     1
Name: Size, Length: 413, dtype: int64

In [229]:
# 'Varies with device'라는 값이 있기 때문에 이걸 nan으로 바뚸준다.
data['Size'] = data['Size'].replace('Varies with device', np.nan)

In [233]:
# 변환한 nan을 각 앱의 카테고리별 평균 값으로 대체한다.
data.groupby(['Category'])['Size'].mean().reset_index()

DataError: No numeric types to aggregate

In [230]:
data['Size'].value_counts()

14000000    165
12000000    161
11000000    159
15000000    159
13000000    157
           ... 
103000        1
939000        1
525000        1
442000        1
787000        1
Name: Size, Length: 412, dtype: int64

### 2.2.5 Installs int 형으로 변환

In [133]:
data['Installs'].unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '100+', '500+', '10+',
       '5+', '50+', '1+'], dtype=object)

In [134]:
data['Installs'] = data['Installs'].str.replace('+', '')
data['Installs'] = data['Installs'].str.replace(',', '')

In [135]:
data['Installs'] = data['Installs'].astype(int)

In [136]:
data['Installs'].unique()

array([     10000,     500000,    5000000,   50000000,     100000,
            50000,    1000000,   10000000,       5000,  100000000,
       1000000000,       1000,  500000000,        100,        500,
               10,          5,         50,          1])

### 2.2.6. Price에서 $빼고 float 형으로 변환

In [127]:
data['Price'].value_counts().head()

0        8715
$2.99     114
$0.99     106
$4.99      70
$1.99      59
Name: Price, dtype: int64

In [128]:
data['Price'] = data['Price'].str.replace('$', '')

In [129]:
data['Price'].value_counts().head()

0       8715
2.99     114
0.99     106
4.99      70
1.99      59
Name: Price, dtype: int64

In [130]:
data['Price'] = data['Price'].astype(float)

In [131]:
data['Price'].describe()

count    9360.000000
mean        0.961279
std        15.821640
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       400.000000
Name: Price, dtype: float64

### 2.2.6. Genres 간략화

In [155]:
# Genres는 Category의 세부분류이거나 중복 카테고리를 표시한 것이지만,
# Category 대부분이 컬럼과 겹친다.
data['Genres'].unique()

array(['Art & Design', 'Art & Design;Pretend Play',
       'Art & Design;Creativity', 'Auto & Vehicles', 'Beauty',
       'Books & Reference', 'Business', 'Comics', 'Comics;Creativity',
       'Communication', 'Dating', 'Education;Education', 'Education',
       'Education;Creativity', 'Education;Music & Video',
       'Education;Action & Adventure', 'Education;Pretend Play',
       'Education;Brain Games', 'Entertainment',
       'Entertainment;Music & Video', 'Entertainment;Brain Games',
       'Entertainment;Creativity', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Lifestyle;Pretend Play',
       'Adventure;Action & Adventure', 'Arcade', 'Casual', 'Card',
       'Casual;Pretend Play', 'Action', 'Strategy', 'Puzzle', 'Sports',
       'Music', 'Word', 'Racing', 'Casual;Creativity',
       'Casual;Action & Adventure', 'Simulation', 'Adventure', 'Board',
       'Trivia', 'Role Playing', 'Simulation;Education',
 

In [156]:
# 또한 세부분류를 한 것들은 각각의 수가 너무 적어서 의미있는 데이터라고 보기 어렵다.
data.Genres.value_counts().tail(15)

Parenting;Brain Games                  1
Strategy;Creativity                    1
Card;Brain Games                       1
Entertainment;Education                1
Racing;Pretend Play                    1
Board;Pretend Play                     1
Role Playing;Brain Games               1
Lifestyle;Education                    1
Adventure;Brain Games                  1
Arcade;Pretend Play                    1
Health & Fitness;Action & Adventure    1
Comics;Creativity                      1
Lifestyle;Pretend Play                 1
Strategy;Education                     1
Tools;Education                        1
Name: Genres, dtype: int64

In [157]:
# 따라서 Genres의 값들을 ; 을 기준으로 나눈 뒤 앞의 값들만 사용한다.
data['Genres'] = data['Genres'].str.split(';').str[0]

In [158]:
data['Genres'].unique()

array(['Art & Design', 'Auto & Vehicles', 'Beauty', 'Books & Reference',
       'Business', 'Comics', 'Communication', 'Dating', 'Education',
       'Entertainment', 'Events', 'Finance', 'Food & Drink',
       'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Adventure', 'Arcade', 'Casual', 'Card', 'Action',
       'Strategy', 'Puzzle', 'Sports', 'Music', 'Word', 'Racing',
       'Simulation', 'Board', 'Trivia', 'Role Playing', 'Educational',
       'Music & Audio', 'Video Players & Editors', 'Medical', 'Social',
       'Shopping', 'Photography', 'Travel & Local', 'Tools',
       'Personalization', 'Productivity', 'Parenting', 'Weather',
       'News & Magazines', 'Maps & Navigation', 'Casino'], dtype=object)

In [159]:
# Music & Audio 은 Music에 포함시키기로 한다.
data.Genres.value_counts().tail(10)

Comics           58
Parenting        50
Card             48
Events           45
Beauty           42
Casino           37
Word             28
Trivia           28
Music            24
Music & Audio     1
Name: Genres, dtype: int64

In [160]:
data['Genres'] = data['Genres'].str.replace('Music & Audio', 'Music')

In [161]:
data.Genres.value_counts().tail(10)

Board        60
Comics       58
Parenting    50
Card         48
Events       45
Beauty       42
Casino       37
Trivia       28
Word         28
Music        25
Name: Genres, dtype: int64

### 2.2.6. Last Updated를 int 형으로 변환
Last Updated는 숫자와 문자가 섞여있어서 바로 datetime 형식으로 바꾸는데 어려움이 있다.<br>
따라서 Last Updated 상의 가장 최근 날짜를 현재날짜라 가정하고, 각 데이터의 Last Updated 날짜에서 현재날짜를 뺌으로서 마지막 업데이트를 한 지 몇 일이 지났는지 계산하는 방식을 이용한다.

In [167]:
data['date'] = pd.to_datetime(data['Last Updated'])
data['date'].describe()

<ipython-input-167-6afdadc00e25>:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data['date'].describe()


count                    9360
unique                   1299
top       2018-08-03 00:00:00
freq                      319
first     2010-05-21 00:00:00
last      2018-08-08 00:00:00
Name: date, dtype: object

In [168]:
data['date'].max()
# 2018-08-03을 현재날짜라고 가정한다.

Timestamp('2018-08-08 00:00:00')

In [169]:
# lastupdate 컬럼을 만들어 계산한 값을 넣는다.
data['lastupdate'] = (data['date'] -  data['date'].max()).dt.days
data['lastupdate'].head()

0   -213
1   -205
2     -7
3    -61
4    -49
Name: lastupdate, dtype: int64

In [170]:
# date는 지워준다.
data = data.drop(labels = ['date'], axis = 1)

# Last Updated도 지운다.
data = data.drop(labels = ['Last Updated'], axis = 1)

### 2.2.6. 원핫 인코딩

In [171]:
data.dtypes

Category           object
Rating            float64
Reviews             int32
Size               object
Installs           object
Type               object
Price             float64
Content Rating     object
Genres             object
lastupdate          int64
dtype: object

In [188]:
data['Size'].value_counts().head()

Varies with device    1637
14M                    165
12M                    161
11M                    159
15M                    159
Name: Size, dtype: int64

In [106]:
# ata = pd.get_dummies(data, columns = ['Category', 'Size', 'Installs', 'Type', 'Content Rating', 'Genres'])
# data.head()

,Rating,Reviews,lastupdate,Category_ART_AND_DESIGN,Category_AUTO_AND_VEHICLES,Category_BEAUTY,Category_BOOKS_AND_REFERENCE,Category_BUSINESS,Category_COMICS,Category_COMMUNICATION,...,Genres_Simulation,Genres_Social,Genres_Sports,Genres_Strategy,Genres_Tools,Genres_Travel & Local,Genres_Trivia,Genres_Video Players & Editors,Genres_Weather,Genres_Word
0,4.1,159,-213,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3.9,967,-205,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4.7,87510,-7,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.5,215644,-61,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4.3,967,-49,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
